# The packet lifetime

Packets are typically created by the application and sent to the operating system, who manages the network stack, via the socket API. For every network layer the packet travel towards the physical medium for transmission, additional information gets prepended or appended into the packet.

The following figure shows the behavior.
![](http://www.tcpipguide.com/free/diagrams/osipdusdu.png)

After the packet gets transmitted via the physical medium, it is received by its peer on the other side of the channel, which decodes that packet and progressively removes the prepended/appended information to verify if it is the final destination of the packet. In case it isn't, it can retransmit to a different peer via the link layer forwarding.

In case it is the final destination, another header is removed from the prepended data, identifying the application which should received the said package.

In any case, the actual packet lifetime is hidden away from the user, since the OS implements the functionality.

Since we have ns-3, we can track the lifetime of the packets and how they travel throughout the stack.

To do that, we need some patches to the upstream code. The packets simply export the program stack every time the simulator creates or copies a new packet, which allows us to track which section of the code resulted in a packet creation and copy.

Here is an example of such stacks.

```
0
0
 0# boost::stacktrace::basic_stacktrace<std::allocator<boost::stacktrace::frame> >::init(unsigned long long, unsigned long long) in libns3_dev_network_debug
 1# ns3::Packet::Packet() in libns3_dev_network_debug
 2# std::__cxx11::list<ns3::GtpcCreateSessionRequestMessage::BearerContextToBeCreated, std::allocator<ns3::GtpcCreateSessionRequestMessage::BearerContextToBeCreated> >::size() const in libns3_dev_lte_debug
 3# ns3::EpcMmeApplication::DoInitialUeMessage(unsigned long long, unsigned short, unsigned long long, unsigned short) in libns3_dev_lte_debug
 4# ns3::MemberEpcS1apSapMme<ns3::EpcMmeApplication>::InitialUeMessage(unsigned long long, unsigned short, unsigned long long, unsigned short) in libns3_dev_lte_debug
 5# ns3::EpcEnbApplication::DoInitialUeMessage(unsigned long long, unsigned short) in libns3_dev_lte_debug
 6# ns3::MemberEpcEnbS1SapProvider<ns3::EpcEnbApplication>::InitialUeMessage(unsigned long long, unsigned short) in libns3_dev_lte_debug
 7# ns3::UeManager::RecvRrcConnectionSetupCompleted(ns3::LteRrcSap::RrcConnectionSetupCompleted) in libns3_dev_lte_debug
 8# ns3::LteEnbRrc::DoRecvRrcConnectionSetupCompleted(unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted) in libns3_dev_lte_debug
 9# ns3::MakeEvent<void (ns3::LteEnbRrc::*)(unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted), ns3::LteEnbRrc*, unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted>(void (ns3::LteEnbRrc::*)(unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted), ns3::LteEnbRrc*, unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted)::EventMemberImpl2::Notify() in libns3_dev_lte_debug
10# ns3::EventImpl::Invoke() in libns3_dev_core_debug
11# ns3::DefaultSimulatorImpl::ProcessOneEvent() in libns3_dev_core_debug
12# ns3::DefaultSimulatorImpl::Run() in libns3_dev_core_debug
13# ns3::Simulator::Run() in libns3_dev_core_debug
14# 0x00007FF6830336CA in ns3_dev_lena_simple_epc_debug
15# 0x00007FF68303143F in ns3_dev_lena_simple_epc_debug
16# 0x00007FF683031146 in ns3_dev_lena_simple_epc_debug
17# _BaseThreadInitThunk in KERNEL32
18# _RtlUserThreadStart in ntdll


0
1
 0# boost::stacktrace::basic_stacktrace<std::allocator<boost::stacktrace::frame> >::init(unsigned long long, unsigned long long) in libns3_dev_network_debug
 1# ns3::Packet::Copy() const in libns3_dev_network_debug
 2# ns3::UdpSocketImpl::DoSendTo(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, unsigned short, unsigned char) in libns3_dev_internet_debug
 3# ns3::UdpSocketImpl::SendTo(ns3::Ptr<ns3::Packet>, unsigned int, ns3::Address const&) in libns3_dev_internet_debug
 4# ns3::EpcMmeApplication::DoInitialUeMessage(unsigned long long, unsigned short, unsigned long long, unsigned short) in libns3_dev_lte_debug
 5# ns3::MemberEpcS1apSapMme<ns3::EpcMmeApplication>::InitialUeMessage(unsigned long long, unsigned short, unsigned long long, unsigned short) in libns3_dev_lte_debug
 6# ns3::EpcEnbApplication::DoInitialUeMessage(unsigned long long, unsigned short) in libns3_dev_lte_debug
 7# ns3::MemberEpcEnbS1SapProvider<ns3::EpcEnbApplication>::InitialUeMessage(unsigned long long, unsigned short) in libns3_dev_lte_debug
 8# ns3::UeManager::RecvRrcConnectionSetupCompleted(ns3::LteRrcSap::RrcConnectionSetupCompleted) in libns3_dev_lte_debug
 9# ns3::LteEnbRrc::DoRecvRrcConnectionSetupCompleted(unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted) in libns3_dev_lte_debug
10# ns3::MakeEvent<void (ns3::LteEnbRrc::*)(unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted), ns3::LteEnbRrc*, unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted>(void (ns3::LteEnbRrc::*)(unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted), ns3::LteEnbRrc*, unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted)::EventMemberImpl2::Notify() in libns3_dev_lte_debug
11# ns3::EventImpl::Invoke() in libns3_dev_core_debug
12# ns3::DefaultSimulatorImpl::ProcessOneEvent() in libns3_dev_core_debug
13# ns3::DefaultSimulatorImpl::Run() in libns3_dev_core_debug
14# ns3::Simulator::Run() in libns3_dev_core_debug
15# 0x00007FF6830336CA in ns3_dev_lena_simple_epc_debug
16# 0x00007FF68303143F in ns3_dev_lena_simple_epc_debug
17# 0x00007FF683031146 in ns3_dev_lena_simple_epc_debug
18# _BaseThreadInitThunk in KERNEL32
19# _RtlUserThreadStart in ntdll


1
2
 0# boost::stacktrace::basic_stacktrace<std::allocator<boost::stacktrace::frame> >::init(unsigned long long, unsigned long long) in libns3_dev_network_debug
 1# ns3::Packet::Copy() const in libns3_dev_network_debug
 2# ns3::Ipv4L3Protocol::Send(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>) in libns3_dev_internet_debug
 3# void std::__invoke_impl<void, void (ns3::Ipv4::*&)(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>), ns3::Ptr<ns3::Ipv4>, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route> >(std::__invoke_memfun_deref, void (ns3::Ipv4::*&)(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>), ns3::Ptr<ns3::Ipv4>&&, ns3::Ptr<ns3::Packet>&&, ns3::Ipv4Address&&, ns3::Ipv4Address&&, unsigned char&&, ns3::Ptr<ns3::Ipv4Route>&&) in libns3_dev_internet_debug
 4# std::enable_if<is_invocable_r_v<void, void (ns3::Ipv4::*&)(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>), ns3::Ptr<ns3::Ipv4>, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route> >, void>::type std::__invoke_r<void, void (ns3::Ipv4::*&)(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>), ns3::Ptr<ns3::Ipv4>, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route> >(void (ns3::Ipv4::*&)(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>), ns3::Ptr<ns3::Ipv4>&&, ns3::Ptr<ns3::Packet>&&, ns3::Ipv4Address&&, ns3::Ipv4Address&&, unsigned char&&, ns3::Ptr<ns3::Ipv4Route>&&) in libns3_dev_internet_debug
 5# std::_Function_handler<void (ns3::Ptr<ns3::Ipv4>, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>), void (ns3::Ipv4::*)(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>)>::_M_invoke(std::_Any_data const&, ns3::Ptr<ns3::Ipv4>&&, ns3::Ptr<ns3::Packet>&&, ns3::Ipv4Address&&, ns3::Ipv4Address&&, unsigned char&&, ns3::Ptr<ns3::Ipv4Route>&&) in libns3_dev_internet_debug
 6# std::function<void (ns3::Ptr<ns3::Ipv4>, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>)>::operator()(ns3::Ptr<ns3::Ipv4>, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>) const in libns3_dev_internet_debug
 7# _ZZN3ns38CallbackIvJNS_3PtrINS_6PacketEEENS_11Ipv4AddressES4_hNS1_INS_9Ipv4RouteEEEEEC4IMNS_4Ipv4EFvS3_S4_S4_hS6_ELi0EJNS1_IS9_EEEEET_DpT1_ENKUlDpOT_E_clIJS3_S4_S4_hS6_EEEvSI_ in libns3_dev_internet_debug
 8# _ZSt13__invoke_implIvRZN3ns38CallbackIvJNS0_3PtrINS0_6PacketEEENS0_11Ipv4AddressES5_hNS2_INS0_9Ipv4RouteEEEEEC4IMNS0_4Ipv4EFvS4_S5_S5_hS7_ELi0EJNS2_ISA_EEEEET_DpT1_EUlDpOT_E_JS4_S5_S5_hS7_EESE_St14__invoke_otherOT0_DpOSF_ in libns3_dev_internet_debug
 9# _ZSt10__invoke_rIvRZN3ns38CallbackIvJNS0_3PtrINS0_6PacketEEENS0_11Ipv4AddressES5_hNS2_INS0_9Ipv4RouteEEEEEC4IMNS0_4Ipv4EFvS4_S5_S5_hS7_ELi0EJNS2_ISA_EEEEET_DpT1_EUlDpOT_E_JS4_S5_S5_hS7_EENSt9enable_ifIX16is_invocable_r_vISE_T0_SG_EESE_E4typeEOSN_DpOSF_ in libns3_dev_internet_debug
10# _ZNSt17_Function_handlerIFvN3ns33PtrINS0_6PacketEEENS0_11Ipv4AddressES4_hNS1_INS0_9Ipv4RouteEEEEZNS0_8CallbackIvJS3_S4_S4_hS6_EEC4IMNS0_4Ipv4EFvS3_S4_S4_hS6_ELi0EJNS1_ISB_EEEEET_DpT1_EUlDpOT_E_E9_M_invokeERKSt9_Any_dataOS3_OS4_SR_OhOS6_ in libns3_dev_internet_debug
11# std::function<void (ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>)>::operator()(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>) const in libns3_dev_internet_debug
12# ns3::CallbackImpl<void, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route> >::operator()(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>) const in libns3_dev_internet_debug
13# ns3::Callback<void, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route> >::operator()(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>) const in libns3_dev_internet_debug
14# ns3::UdpL4Protocol::Send(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned short, unsigned short, ns3::Ptr<ns3::Ipv4Route>) in libns3_dev_internet_debug
15# ns3::UdpSocketImpl::DoSendTo(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, unsigned short, unsigned char) in libns3_dev_internet_debug
16# ns3::UdpSocketImpl::SendTo(ns3::Ptr<ns3::Packet>, unsigned int, ns3::Address const&) in libns3_dev_internet_debug
17# ns3::EpcMmeApplication::DoInitialUeMessage(unsigned long long, unsigned short, unsigned long long, unsigned short) in libns3_dev_lte_debug
18# ns3::MemberEpcS1apSapMme<ns3::EpcMmeApplication>::InitialUeMessage(unsigned long long, unsigned short, unsigned long long, unsigned short) in libns3_dev_lte_debug
19# ns3::EpcEnbApplication::DoInitialUeMessage(unsigned long long, unsigned short) in libns3_dev_lte_debug
20# ns3::MemberEpcEnbS1SapProvider<ns3::EpcEnbApplication>::InitialUeMessage(unsigned long long, unsigned short) in libns3_dev_lte_debug
21# ns3::UeManager::RecvRrcConnectionSetupCompleted(ns3::LteRrcSap::RrcConnectionSetupCompleted) in libns3_dev_lte_debug
22# ns3::LteEnbRrc::DoRecvRrcConnectionSetupCompleted(unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted) in libns3_dev_lte_debug
23# ns3::MakeEvent<void (ns3::LteEnbRrc::*)(unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted), ns3::LteEnbRrc*, unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted>(void (ns3::LteEnbRrc::*)(unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted), ns3::LteEnbRrc*, unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted)::EventMemberImpl2::Notify() in libns3_dev_lte_debug
24# ns3::EventImpl::Invoke() in libns3_dev_core_debug
25# ns3::DefaultSimulatorImpl::ProcessOneEvent() in libns3_dev_core_debug
26# ns3::DefaultSimulatorImpl::Run() in libns3_dev_core_debug
27# ns3::Simulator::Run() in libns3_dev_core_debug
28# 0x00007FF6830336CA in ns3_dev_lena_simple_epc_debug
29# 0x00007FF68303143F in ns3_dev_lena_simple_epc_debug
30# 0x00007FF683031146 in ns3_dev_lena_simple_epc_debug
31# _BaseThreadInitThunk in KERNEL32
32# _RtlUserThreadStart in ntdll


2
3
 0# boost::stacktrace::basic_stacktrace<std::allocator<boost::stacktrace::frame> >::init(unsigned long long, unsigned long long) in libns3_dev_network_debug
 1# ns3::Packet::Copy() const in libns3_dev_network_debug
 2# ns3::Ipv4L3Protocol::Send(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>) in libns3_dev_internet_debug
 3# ns3::Ipv4L3Protocol::Send(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>) in libns3_dev_internet_debug
 4# void std::__invoke_impl<void, void (ns3::Ipv4::*&)(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>), ns3::Ptr<ns3::Ipv4>, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route> >(std::__invoke_memfun_deref, void (ns3::Ipv4::*&)(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>), ns3::Ptr<ns3::Ipv4>&&, ns3::Ptr<ns3::Packet>&&, ns3::Ipv4Address&&, ns3::Ipv4Address&&, unsigned char&&, ns3::Ptr<ns3::Ipv4Route>&&) in libns3_dev_internet_debug
 5# std::enable_if<is_invocable_r_v<void, void (ns3::Ipv4::*&)(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>), ns3::Ptr<ns3::Ipv4>, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route> >, void>::type std::__invoke_r<void, void (ns3::Ipv4::*&)(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>), ns3::Ptr<ns3::Ipv4>, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route> >(void (ns3::Ipv4::*&)(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>), ns3::Ptr<ns3::Ipv4>&&, ns3::Ptr<ns3::Packet>&&, ns3::Ipv4Address&&, ns3::Ipv4Address&&, unsigned char&&, ns3::Ptr<ns3::Ipv4Route>&&) in libns3_dev_internet_debug
 6# std::_Function_handler<void (ns3::Ptr<ns3::Ipv4>, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>), void (ns3::Ipv4::*)(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>)>::_M_invoke(std::_Any_data const&, ns3::Ptr<ns3::Ipv4>&&, ns3::Ptr<ns3::Packet>&&, ns3::Ipv4Address&&, ns3::Ipv4Address&&, unsigned char&&, ns3::Ptr<ns3::Ipv4Route>&&) in libns3_dev_internet_debug
 7# std::function<void (ns3::Ptr<ns3::Ipv4>, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>)>::operator()(ns3::Ptr<ns3::Ipv4>, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>) const in libns3_dev_internet_debug
 8# _ZZN3ns38CallbackIvJNS_3PtrINS_6PacketEEENS_11Ipv4AddressES4_hNS1_INS_9Ipv4RouteEEEEEC4IMNS_4Ipv4EFvS3_S4_S4_hS6_ELi0EJNS1_IS9_EEEEET_DpT1_ENKUlDpOT_E_clIJS3_S4_S4_hS6_EEEvSI_ in libns3_dev_internet_debug
 9# _ZSt13__invoke_implIvRZN3ns38CallbackIvJNS0_3PtrINS0_6PacketEEENS0_11Ipv4AddressES5_hNS2_INS0_9Ipv4RouteEEEEEC4IMNS0_4Ipv4EFvS4_S5_S5_hS7_ELi0EJNS2_ISA_EEEEET_DpT1_EUlDpOT_E_JS4_S5_S5_hS7_EESE_St14__invoke_otherOT0_DpOSF_ in libns3_dev_internet_debug
10# _ZSt10__invoke_rIvRZN3ns38CallbackIvJNS0_3PtrINS0_6PacketEEENS0_11Ipv4AddressES5_hNS2_INS0_9Ipv4RouteEEEEEC4IMNS0_4Ipv4EFvS4_S5_S5_hS7_ELi0EJNS2_ISA_EEEEET_DpT1_EUlDpOT_E_JS4_S5_S5_hS7_EENSt9enable_ifIX16is_invocable_r_vISE_T0_SG_EESE_E4typeEOSN_DpOSF_ in libns3_dev_internet_debug
11# _ZNSt17_Function_handlerIFvN3ns33PtrINS0_6PacketEEENS0_11Ipv4AddressES4_hNS1_INS0_9Ipv4RouteEEEEZNS0_8CallbackIvJS3_S4_S4_hS6_EEC4IMNS0_4Ipv4EFvS3_S4_S4_hS6_ELi0EJNS1_ISB_EEEEET_DpT1_EUlDpOT_E_E9_M_invokeERKSt9_Any_dataOS3_OS4_SR_OhOS6_ in libns3_dev_internet_debug
12# std::function<void (ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>)>::operator()(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>) const in libns3_dev_internet_debug
13# ns3::CallbackImpl<void, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route> >::operator()(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>) const in libns3_dev_internet_debug
14# ns3::Callback<void, ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route> >::operator()(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned char, ns3::Ptr<ns3::Ipv4Route>) const in libns3_dev_internet_debug
15# ns3::UdpL4Protocol::Send(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, ns3::Ipv4Address, unsigned short, unsigned short, ns3::Ptr<ns3::Ipv4Route>) in libns3_dev_internet_debug
16# ns3::UdpSocketImpl::DoSendTo(ns3::Ptr<ns3::Packet>, ns3::Ipv4Address, unsigned short, unsigned char) in libns3_dev_internet_debug
17# ns3::UdpSocketImpl::SendTo(ns3::Ptr<ns3::Packet>, unsigned int, ns3::Address const&) in libns3_dev_internet_debug
18# ns3::EpcMmeApplication::DoInitialUeMessage(unsigned long long, unsigned short, unsigned long long, unsigned short) in libns3_dev_lte_debug
19# ns3::MemberEpcS1apSapMme<ns3::EpcMmeApplication>::InitialUeMessage(unsigned long long, unsigned short, unsigned long long, unsigned short) in libns3_dev_lte_debug
20# ns3::EpcEnbApplication::DoInitialUeMessage(unsigned long long, unsigned short) in libns3_dev_lte_debug
21# ns3::MemberEpcEnbS1SapProvider<ns3::EpcEnbApplication>::InitialUeMessage(unsigned long long, unsigned short) in libns3_dev_lte_debug
22# ns3::UeManager::RecvRrcConnectionSetupCompleted(ns3::LteRrcSap::RrcConnectionSetupCompleted) in libns3_dev_lte_debug
23# ns3::LteEnbRrc::DoRecvRrcConnectionSetupCompleted(unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted) in libns3_dev_lte_debug
24# ns3::MakeEvent<void (ns3::LteEnbRrc::*)(unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted), ns3::LteEnbRrc*, unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted>(void (ns3::LteEnbRrc::*)(unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted), ns3::LteEnbRrc*, unsigned short, ns3::LteRrcSap::RrcConnectionSetupCompleted)::EventMemberImpl2::Notify() in libns3_dev_lte_debug
25# ns3::EventImpl::Invoke() in libns3_dev_core_debug
26# ns3::DefaultSimulatorImpl::ProcessOneEvent() in libns3_dev_core_debug
27# ns3::DefaultSimulatorImpl::Run() in libns3_dev_core_debug
28# ns3::Simulator::Run() in libns3_dev_core_debug
29# 0x00007FF6830336CA in ns3_dev_lena_simple_epc_debug
30# 0x00007FF68303143F in ns3_dev_lena_simple_epc_debug
31# 0x00007FF683031146 in ns3_dev_lena_simple_epc_debug
32# _BaseThreadInitThunk in KERNEL32
33# _RtlUserThreadStart in ntdll
```

After running a collating script, we can generate a diagram.

In the meantime, I have used MermaidJS for ease of use.

[![](https://mermaid.ink/img/pako:eNqNm0-T27gVxL-Ki2fbhffwfw455bgH1eaWTA6KR4634tFMjWdqs3H5u0eiCKHx1iL7JIJiN0DgR0BUk9-nT08Ph-lu-vz16fdPX_Yvr-9--fX-uHP3x29v__r3y_75y7v7af_8fD_dHw9fD4_uH-_-efm-lT58-MtcPD4Mmsf9p6aRQSMXTTeblTsZ1K8v--O3a6V6MZBeOlcqf670eHj9_enlP03nB53vFWuv-GalYRCHLvY_FUMfxUEZuzL8VLn01Ll_zpqdW46O0Du6dprpUp_20rl7dOsE8yDLvZnpp800lZZBXbo6E31bB3Ht4sJULW6Qi-v6SlQuMsoBSHFbYys6ihXEsjq8clHtpB0OEO786vkuHHsonkfYb55pGIUAsXiqp-NoACxLYLo6jfoE-kg1II8GQKkkpgFl1AOnkjfHuo5i4FTK6ljrRbXTdjhwuQtrJ6wL2wGK57EOW2eqMgqBanVMV6uOBkC2CtHV6kc9TrdKNSCMBkCreqYBcdQDrBq2xlrTKAZSNa6OdYBxVmByF1dPdgE7QvE8znHzLMsoBKI1U91cRwOgWgvRzd4Neg-zr1amAV5GAyDVO6YBOuoBVC9b4-z9KAZKvW6Oczx_tsOByV26Wd8CdYLieZzT6i8mH0cVoOwB5V1e66aF6AzFc815e4DyqIRJ1zOTri-jHhD1m5Our6MY8PTrk266qHapHY6Tblk74bAgXaB47qqy-RtRRiGgHKhJN-hoACwHZtINftQDzoGadEMYDWDSDcykG-Kox1-7m5NuSKMYJt2wPunmi2qX2-E48dbVE17grlA8j3XdPNMyCoHqQE28oY4GQHZgJt7oBn2EiTdQE2-U0QBojczEG3XUA6xxc-KNfhQDqXF94i0X1a60w3HiFbd6xu0mzkF5vnd0mycbjRTIjoHq7mQcAO8Ymf7OxgAm4kjdpMViHADayNyoxWoMANpYtsY8uVGdANlY1--WHIx4Gv4okPb1sjWP57ypyxJ92Zj3z3dcN--u5q-WI_21grB--Pn709YsCldRvC0Ky_enrVkUr6K00vVpuVolQXnWp9Vxn7-_VAU2S4X55lC1_1gylOfa8hYjyRspXNqJWoRSMA6wCiVmFUrRGMDFmqiLNSXjABdrYi7WlI0B_qFCXaypGAe4WBNzsaZqDOBiTdQ_K9mNDhku2MT8t5LFGMAak6lfRNmAmGGVycxPomxwzIBjpnDMBscMOGYGx2xwzIBjpnDMBscMOGYGx2xwzIBjpnDMBseM__ExOGaDYwYcM4VjMTgWwDEzOBaDYwEcC4VjMTgWwLEMU2tZa0djskB5nl3LdguC0QKNhaGxRGMANJbN3-glGTWQWBgSSzYGQGLh_mouxgFILAyJpRoD_MeZIrG60aECiYUhsYoxABIrRWJV4wAkVmZirAbCChNjpSbGalCsgGJlUKwGxQooVmpirAbHCjhWBsdqcKyAY6VwrAbHCjhWKv0wOFbAsZIJiOHxsqN5UCmIE2sxBCGOa4daEwxEnFAN8dbDo4dyDQnWBMMO56mGROuBeYfb5HNeJGb1aesqwxhP6vpJNLgr7pjXibrd-mzFGJa4xHVjsSaYmLhMdWO1HhicOA7wlvFdTTDla1Vs5XxiPZBwIutrYV83GOI-iu4W3nUPpFs4uluQ102QbqHoblle90C6JXANsYQOkZ5EqiEW1CHVEw5UsaAO0Z5QoIoFdUj4hANVLaiKoAoFqlpQMb0TLr4TtbBigidUhCdqYcUUT7gYT9TCikmeUFGeqIUV0zxRDla1sGKqJ0rBqhZWRViVg1UtrJjaiVKwqoUVgztRDlZvYcX0TpSC1VtYMcAT77ZXybKoT1tXGd5KqVs_ifYIhsMd8yMvbrv13oqRcM8R3kK9bjI8WUER3iK-7oGEe47wlvF1EyTcU4S3tK97IOE3Aj9cJVvc1w2Qbk_R3VK_7oF0e47uFuNdTQLS7Sm6W6LXPZBuLtSTYAnFXE-oYE-CBRWzPeHCPQkWVMz3hAr4JFhQw_AUEAdqsKBi1ieBAjVYUAOCGripOFhYMceTQMEaLKwY5UngYI0WVszzJFCwRgsrRnoSOVijhRVzPYkUrNHCivGeRA7WaGGNCGukYI0WVozthMvtJFpY4_DcGgVrtLBieCfb6d1lgdQ55qpXWcZVUtZPooEuuGNeJWW79dWKkfDIEd7Sv6sJxn8SKcJbCNY9kPDEEd6yrW6ChCeK8BZydQ8k_EbMhatky7i6AdJNpVzSYq7ugXRzQZe0pKubIN1U1iUt7Ooew4OZ3FScLKEYeAmVeEmyoGLmJVzoJdmCirGXULmXZAsqJl_CRV-SLagYfgmVfkm2oGL-JVwAJtnCihGYUBmYZAsrpmDCxWCSLawYhAmVhEm2sGIWJlwYJtnCmodHiSlYs4UVEzHhIjEpFlYMxYRKxaRYWDEXEy4Yk2JhxWhMCgVrsbAWhLVswjoLYYHEPGynut7-xrjijnmB1O2GRytGuAsHd4vWugnCTYVr0tK17oFwc_matICtmyDcVMQmLWPrHsPj8mX7YXtnDDBjEypkk5aydQ8Em8vZpAVt3QTBpqI2aVlb90CwubRNqiUU8zahAjepFlSM3ITL3KRaUDF1Eyp2k2pBxeBNuORNqgUVszehwjepFlSM34TL39QZWBXzN6ECOHViPYZ3NriXNpxaE3xvgwrg1Hnrge9ucAGcumBN8P0NKoBTF60HvsPhAteQZE3wXQ4XqYZk65HRI3ENKdYE39WgsjR11Xrg6xrbWdplgTyrT1tXGT4Yr749SblszSvg_PLS7ecgNfRDr49Aalw__vz9aWsWXR-B1HRbFJfvT1uzKF1FefVFqHZtZtwxO-SNzsqtMjQ6Kab30-Ph5XH_28N0N32_P7471_rl8Hi4n-5Omw-Hz_u3r69njx-nQ_dvr09_--P4abr7vP_67fB-ent-2L8e_vrb_lTp43Xv8_7496enU_n15e1SnO6-T_-d7j5I8OljiOqjL-H0e8yn99Mf0130H2M63a_6WHMsRWL68X7632zhP8rpUK2nVcG54H348X89hjmU?type=png)](https://mermaid.live/edit#pako:eNqNm0-T27gVxL-Ki2fbhffwfw455bgH1eaWTA6KR4634tFMjWdqs3H5u0eiCKHx1iL7JIJiN0DgR0BUk9-nT08Ph-lu-vz16fdPX_Yvr-9--fX-uHP3x29v__r3y_75y7v7af_8fD_dHw9fD4_uH-_-efm-lT58-MtcPD4Mmsf9p6aRQSMXTTeblTsZ1K8v--O3a6V6MZBeOlcqf670eHj9_enlP03nB53vFWuv-GalYRCHLvY_FUMfxUEZuzL8VLn01Ll_zpqdW46O0Du6dprpUp_20rl7dOsE8yDLvZnpp800lZZBXbo6E31bB3Ht4sJULW6Qi-v6SlQuMsoBSHFbYys6ihXEsjq8clHtpB0OEO786vkuHHsonkfYb55pGIUAsXiqp-NoACxLYLo6jfoE-kg1II8GQKkkpgFl1AOnkjfHuo5i4FTK6ljrRbXTdjhwuQtrJ6wL2wGK57EOW2eqMgqBanVMV6uOBkC2CtHV6kc9TrdKNSCMBkCreqYBcdQDrBq2xlrTKAZSNa6OdYBxVmByF1dPdgE7QvE8znHzLMsoBKI1U91cRwOgWgvRzd4Neg-zr1amAV5GAyDVO6YBOuoBVC9b4-z9KAZKvW6Oczx_tsOByV26Wd8CdYLieZzT6i8mH0cVoOwB5V1e66aF6AzFc815e4DyqIRJ1zOTri-jHhD1m5Our6MY8PTrk266qHapHY6Tblk74bAgXaB47qqy-RtRRiGgHKhJN-hoACwHZtINftQDzoGadEMYDWDSDcykG-Kox1-7m5NuSKMYJt2wPunmi2qX2-E48dbVE17grlA8j3XdPNMyCoHqQE28oY4GQHZgJt7oBn2EiTdQE2-U0QBojczEG3XUA6xxc-KNfhQDqXF94i0X1a60w3HiFbd6xu0mzkF5vnd0mycbjRTIjoHq7mQcAO8Ymf7OxgAm4kjdpMViHADayNyoxWoMANpYtsY8uVGdANlY1--WHIx4Gv4okPb1sjWP57ypyxJ92Zj3z3dcN--u5q-WI_21grB--Pn709YsCldRvC0Ky_enrVkUr6K00vVpuVolQXnWp9Vxn7-_VAU2S4X55lC1_1gylOfa8hYjyRspXNqJWoRSMA6wCiVmFUrRGMDFmqiLNSXjABdrYi7WlI0B_qFCXaypGAe4WBNzsaZqDOBiTdQ_K9mNDhku2MT8t5LFGMAak6lfRNmAmGGVycxPomxwzIBjpnDMBscMOGYGx2xwzIBjpnDMBscMOGYGx2xwzIBjpnDMBseM__ExOGaDYwYcM4VjMTgWwDEzOBaDYwEcC4VjMTgWwLEMU2tZa0djskB5nl3LdguC0QKNhaGxRGMANJbN3-glGTWQWBgSSzYGQGLh_mouxgFILAyJpRoD_MeZIrG60aECiYUhsYoxABIrRWJV4wAkVmZirAbCChNjpSbGalCsgGJlUKwGxQooVmpirAbHCjhWBsdqcKyAY6VwrAbHCjhWKv0wOFbAsZIJiOHxsqN5UCmIE2sxBCGOa4daEwxEnFAN8dbDo4dyDQnWBMMO56mGROuBeYfb5HNeJGb1aesqwxhP6vpJNLgr7pjXibrd-mzFGJa4xHVjsSaYmLhMdWO1HhicOA7wlvFdTTDla1Vs5XxiPZBwIutrYV83GOI-iu4W3nUPpFs4uluQ102QbqHoblle90C6JXANsYQOkZ5EqiEW1CHVEw5UsaAO0Z5QoIoFdUj4hANVLaiKoAoFqlpQMb0TLr4TtbBigidUhCdqYcUUT7gYT9TCikmeUFGeqIUV0zxRDla1sGKqJ0rBqhZWRViVg1UtrJjaiVKwqoUVgztRDlZvYcX0TpSC1VtYMcAT77ZXybKoT1tXGd5KqVs_ifYIhsMd8yMvbrv13oqRcM8R3kK9bjI8WUER3iK-7oGEe47wlvF1EyTcU4S3tK97IOE3Aj9cJVvc1w2Qbk_R3VK_7oF0e47uFuNdTQLS7Sm6W6LXPZBuLtSTYAnFXE-oYE-CBRWzPeHCPQkWVMz3hAr4JFhQw_AUEAdqsKBi1ieBAjVYUAOCGripOFhYMceTQMEaLKwY5UngYI0WVszzJFCwRgsrRnoSOVijhRVzPYkUrNHCivGeRA7WaGGNCGukYI0WVozthMvtJFpY4_DcGgVrtLBieCfb6d1lgdQ55qpXWcZVUtZPooEuuGNeJWW79dWKkfDIEd7Sv6sJxn8SKcJbCNY9kPDEEd6yrW6ChCeK8BZydQ8k_EbMhatky7i6AdJNpVzSYq7ugXRzQZe0pKubIN1U1iUt7Ooew4OZ3FScLKEYeAmVeEmyoGLmJVzoJdmCirGXULmXZAsqJl_CRV-SLagYfgmVfkm2oGL-JVwAJtnCihGYUBmYZAsrpmDCxWCSLawYhAmVhEm2sGIWJlwYJtnCmodHiSlYs4UVEzHhIjEpFlYMxYRKxaRYWDEXEy4Yk2JhxWhMCgVrsbAWhLVswjoLYYHEPGynut7-xrjijnmB1O2GRytGuAsHd4vWugnCTYVr0tK17oFwc_matICtmyDcVMQmLWPrHsPj8mX7YXtnDDBjEypkk5aydQ8Em8vZpAVt3QTBpqI2aVlb90CwubRNqiUU8zahAjepFlSM3ITL3KRaUDF1Eyp2k2pBxeBNuORNqgUVszehwjepFlSM34TL39QZWBXzN6ECOHViPYZ3NriXNpxaE3xvgwrg1Hnrge9ucAGcumBN8P0NKoBTF60HvsPhAteQZE3wXQ4XqYZk65HRI3ENKdYE39WgsjR11Xrg6xrbWdplgTyrT1tXGT4Yr749SblszSvg_PLS7ecgNfRDr49Aalw__vz9aWsWXR-B1HRbFJfvT1uzKF1FefVFqHZtZtwxO-SNzsqtMjQ6Kab30-Ph5XH_28N0N32_P7471_rl8Hi4n-5Omw-Hz_u3r69njx-nQ_dvr09_--P4abr7vP_67fB-ent-2L8e_vrb_lTp43Xv8_7496enU_n15e1SnO6-T_-d7j5I8OljiOqjL-H0e8yn99Mf0130H2M63a_6WHMsRWL68X7632zhP8rpUK2nVcG54H348X89hjmU)